# Uczenie maszynowe: Lab4

### Inżynieria cech
Inżynieria cech to proces przekształcania danych w celu poprawy wydajności modeli uczenia maszynowego. Obejmuje to zarówno redukcję wymiarowości (np. metodą PCA), jak i selekcję cech, aby wybrać najbardziej informacyjne cechy.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.exceptions import ConvergenceWarning
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel, SelectKBest, f_classif
from sklearn.datasets import load_digits


#### Selekcja cech (różne algorytmy).
Selekcja cech polega na wyborze podzbioru cech, które są najbardziej istotne dla danego problemu predykcyjnego. Można to zrobić na różne sposoby, np. na podstawie znaczenia cech określonego przez model lub poprzez univariate selection, gdzie każda cecha jest oceniana indywidualnie.
- Celem jest zredukowanie liczby cech do tych, które są najbardziej informatywne i istotne dla modelu, co może prowadzić do poprawy jego wydajności, uproszczenia modelu i zmniejszenia ryzyka przeuczenia (overfitting).

In [2]:
warnings.filterwarnings('ignore', category=ConvergenceWarning)
# Dane
data = load_digits()

X = data.data 
y = data.target 

# Usunięcie stałych cech
X = X[:, X.var(axis=0) > 0]

# Trenowanie modelu
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standaryzacja danych
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Tworzenie modelu Random Forest
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Trening modelu bez selekcji cech
model = SVC(kernel='linear', random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność na zbiorze testowym (bez selekcji): {accuracy:.4f}")
print("Raport klasyfikacji (bez selekcji):\n", classification_report(y_test, y_pred, zero_division=1))

# Selekcja cech na podstawie znaczenia cech
selector = SelectFromModel(rf, prefit=True) # wybiera cechy, które mają znaczenie # prefit = True, ponieważ model jest już wytrenowany
X_train_rf = selector.transform(X_train)
X_test_rf = selector.transform(X_test)

# Trening modelu z selekcją cech (znaczenie cech)
model.fit(X_train_rf, y_train)
y_pred_rf = model.predict(X_test_rf)
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Dokładność na zbiorze testowym (znaczenie cech): {accuracy_rf:.4f}")
print("Raport klasyfikacji (znaczenie cech):\n", classification_report(y_test, y_pred_rf, zero_division=1))

# Wybieranie najlepszych cech za pomocą testu ANOVA
# k - liczba cech do wybrania
# score_func - funkcja oceny cech
selector = SelectKBest(score_func=f_classif, k=2) #  wybiera k cech, które mają najwyższe wyniki dla wybranego testu statystycznego, 
# mierząc ich zależność z docelową zmienną
X_train_best = selector.fit_transform(X_train, y_train)
X_test_best = selector.transform(X_test)

# Trenowanie modelu po selekcji cech
model.fit(X_train_best, y_train)
y_pred_best = model.predict(X_test_best)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f"Dokładność na zbiorze testowym (Univariate Selection): {accuracy_best:.4f}")
print("Raport klasyfikacji (Univariate Selection):\n", classification_report(y_test, y_pred_best, zero_division=1))

Dokładność na zbiorze testowym (bez selekcji): 0.9778
Raport klasyfikacji (bez selekcji):
               precision    recall  f1-score   support

           0       0.98      1.00      0.99        53
           1       0.98      0.98      0.98        50
           2       0.98      1.00      0.99        47
           3       0.96      0.96      0.96        54
           4       1.00      1.00      1.00        60
           5       0.97      0.95      0.96        66
           6       0.98      0.98      0.98        53
           7       1.00      0.98      0.99        55
           8       0.95      0.95      0.95        43
           9       0.97      0.97      0.97        59

    accuracy                           0.98       540
   macro avg       0.98      0.98      0.98       540
weighted avg       0.98      0.98      0.98       540

Dokładność na zbiorze testowym (znaczenie cech): 0.9741
Raport klasyfikacji (znaczenie cech):
               precision    recall  f1-score   support



### Redukcja wymiarowości: Metoda PCA
Principal Component Analysis (PCA) to statystyczna metoda analizy danych, która przekształca dane o wysokiej wymiarowości na dane o mniejszej liczbie wymiarów. Celem PCA jest zredukowanie liczby wymiarów przy jednoczesnym zachowaniu jak największej ilości informacji zawartej w oryginalnych danych.
- PCA może prowadzić do mniejszej dokładności modelu w porównaniu do modelu trenowanego bez redukcji wymiarowości. Dzieje się tak dlatego, że PCA zmniejsza liczbę cech, co może skutkować utratą istotnych informacji zawartych w danych. W niektórych przypadkach, szczególnie jeśli liczba wybranych komponentów jest zbyt mała, model może nie być w stanie uchwycić wszystkich istotnych wzorców w danych.

In [3]:
# Wczytywanie danych
data = load_digits()

X = data.data 
y = data.target 

# Podział danych na zbiór treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standaryzacja danych -> każda zmienna ma średnią 0 i wariancję 1 aby zmienne występujące w zbiorze danych były tej samej skali
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Trenowanie modelu bez redukcji wymiarowości
# 100 drzew decyzyjnych
model = RandomForestClassifier(n_estimators=100, random_state=42)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Dokładność na zbiorze testowym (bez redukcji): {accuracy:.4f}")
print("Raport klasyfikacji (bez redukcji):\n", classification_report(y_test, y_pred))

# Redukcja wymiarowości za pomocą PCA
pca = PCA(n_components=2) # redukuje do 2 składowych głównych
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)

model.fit(X_train_pca, y_train)
y_pred_pca = model.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test, y_pred_pca)
print(f"\nDokładność na zbiorze testowym (PCA): {accuracy_pca:.4f}")
print("Raport klasyfikacji (PCA):\n", classification_report(y_test, y_pred_pca))

Dokładność na zbiorze testowym (bez redukcji): 0.9722
Raport klasyfikacji (bez redukcji):
               precision    recall  f1-score   support

           0       1.00      0.97      0.98        33
           1       0.97      1.00      0.98        28
           2       1.00      1.00      1.00        33
           3       1.00      0.94      0.97        34
           4       0.98      1.00      0.99        46
           5       0.94      0.96      0.95        47
           6       0.97      0.97      0.97        35
           7       0.97      0.97      0.97        34
           8       0.97      0.97      0.97        30
           9       0.95      0.95      0.95        40

    accuracy                           0.97       360
   macro avg       0.97      0.97      0.97       360
weighted avg       0.97      0.97      0.97       360


Dokładność na zbiorze testowym (PCA): 0.5389
Raport klasyfikacji (PCA):
               precision    recall  f1-score   support

           0       0.